In [1]:
import numpy as np
import random
import re
import time
from IPython.display import clear_output

In [2]:
def colordict(key, string, bright=True):
    str_i = str(string)
    cdict = {
        0: f"\033[{31 if bright else 91}m{str_i}\033[0m",
        1: f"\033[{34 if bright else 94}m{str_i}\033[0m",
        2: f"\033[{32 if bright else 92}m{str_i}\033[0m",
        3: f"\033[{33 if bright else 93}m{str_i}\033[0m",
        4: f"\033[{35 if bright else 95}m{str_i}\033[0m",
        5: f"\033[{36 if bright else 96}m{str_i}\033[0m",
        6: f"\033[{30 if bright else 90}m{str_i}\033[0m",
        7: f"\033[{37 if bright else 97}m{str_i}\033[0m",
    }
    return cdict[key]

def colorname_dict(key):
    cdict = {
        0: f"red",
        1: f"blue",
        2: f"green",
        3: f"yellow",
        4: f"magenta",
        5: f"cyan",
        6: f"black",
        7: f"white",
    }
    return cdict[key]

def make_AP(start, stop, prog_len):
    APs = []
    for i in range(start, stop + 1):
        for step in range(1, stop - start + 1):
            AP = [i + t * step for t in range(prog_len)]
            if AP[-1] <= stop:
                APs.append(AP)
    return APs

def get_monochromatic_APs(bound, prog_len, num_colors, Coloring=None, seed=None):
    if Coloring is None:
        if seed is not None:
            np.random.seed(seed)
        Coloring = np.random.randint(0, num_colors, bound)
    APS = make_AP(1, bound, prog_len)
    mono_APs = []
    for A in APS:
        colors = [Coloring[i - 1] for i in A]
        if len(set(colors)) == 1:
            mono_APs.append(A)
    return Coloring, mono_APs


def display_coloring_with_AP(Coloring, highlighted_AP=[], bound=27, block_size=9, blocks_per_row=1,
                             print_base=True, print_stats=False, lower_bound=1):
    def format_coloring(coloring, bound, blocks_per_row, block_size, lower_bound):
        per_row = blocks_per_row * block_size
        numbers = np.arange(1, bound + 1)
        rows = (bound + per_row - 1) // per_row
        max_log = int(np.floor(np.log10(bound)))
        strings = []

        for i in range(rows):
            string_i = ''
            for j in range(1, per_row + 1):
                k = per_row * i + j
                if k < lower_bound or k > bound:
                    continue
                log_diff = max_log - int(np.floor(np.log10(k)))
                spacing = ' ' * log_diff
                color = coloring[k - 1]
                number = numbers[k - 1]
                if (j - 1) % block_size == 0:
                    string = '  ' + spacing + colordict(color, number, True) + ' '
                else:
                    string = spacing + colordict(color, number, True) + ' '
                strings.append((k, string))
                string_i += string
            if print_base and string_i != '':
                print(string_i)
        return strings

    show_colors = format_coloring(Coloring, bound, blocks_per_row, block_size, lower_bound)

    if not highlighted_AP:
        return  #

    if print_stats:
        print(f"\nAP: {highlighted_AP} (step size = {highlighted_AP[1] - highlighted_AP[0]})\n")

    per_row = blocks_per_row * block_size
    row_count = (bound + per_row - 1) // per_row

    # Recolor based on highlighted_AP
    recolored = []
    for (k, s) in show_colors:
        if k in highlighted_AP:
            match = re.search(r'(\033\[\d+(;\d+)?m\d+\033\[0m)', s)
            if match:
                colored_num = match.group(1)
                prefix = s[:match.start(1)]
                suffix = s[match.end(1):]
                highlighted = f"{prefix}\033[1;4m{colored_num}\033[0m{suffix}"
                recolored.append((k, highlighted))
            else:
                recolored.append((k, f"\033[1;4m{s}\033[0m"))
        else:
            recolored.append((k, s))

    # Reconstruct rows and print
    current_row = []
    for idx, (k, s) in enumerate(recolored):
        current_row.append(s)
        if (k - lower_bound + 1) % (block_size * blocks_per_row) == 0:
            print(''.join(current_row))
            current_row = []
    if current_row:
        print(''.join(current_row))

In [3]:
def analyze_repeated_block_AP(Coloring, bound, block_size, blocks_per_row, num_colors, prog_len=3):
    import numpy as np
    from IPython.display import display

    num_blocks = bound // block_size
    reshaped_Coloring = Coloring.reshape(num_blocks, block_size)
    bound0 = num_colors**block_size + 1  

    # Step 1: Encode block color patterns
    color_patterns = []
    for i in range(num_blocks):
        pattern_str = ''.join(str(x) for x in reshaped_Coloring[i])
        pattern_val = int(pattern_str, num_colors)
        color_patterns.append(pattern_val)
    color_patterns = np.array(color_patterns)

    # Step 2: Identify earliest repeated pattern with both b1, b2 ≤ 32
    repeated_pattern_dict = {}
    unique_patterns = np.unique(color_patterns)
    for p in unique_patterns:
        indices = np.where(color_patterns == p)[0]
        if len(indices) > 1:
            repeated_pattern_dict[p] = indices

    min_index = num_blocks + 1
    min_pattern = None
    for key, val in repeated_pattern_dict.items():
        b1_candidate = val[0]
        b2_candidate = val[1]
        b3_candidate = 2 * b2_candidate - b1_candidate
        if b1_candidate <= 32 and b2_candidate <= 32 and b3_candidate < num_blocks:
            if b1_candidate < min_index:
                min_index = b1_candidate
                min_pattern = key

    if min_pattern is None:
        print("No suitable repeated pattern found within {0,...,32}.")
        return

    # Step 3: Reconstruct block indices
    b1 = repeated_pattern_dict[min_pattern][0]
    b2 = repeated_pattern_dict[min_pattern][1]
    b3 = 2 * b2 - b1

    block_1_nums = [b1 * block_size + i for i in range(1, block_size + 1)]
    block_2_nums = [b2 * block_size + i for i in range(1, block_size + 1)]
    block_3_nums = [b3 * block_size + i for i in range(1, block_size + 1)]

    # Step 4: Identify two positions of same color in first 3 numbers of block b1
    color_count = np.zeros(num_colors)
    same_colored = []

    for i in block_1_nums[:3]:  # only check 5b1 + 1, 2, 3
        color_count[Coloring[i - 1]] += 1
    repeated_color = np.argmax(color_count)

    for i in block_1_nums[:3]:
        if Coloring[i - 1] == repeated_color:
            same_colored.append(i - b1 * block_size)  # in-block position

    if len(same_colored) < 2:
        print("Not enough repeated colors in first 3 positions of block.")
        return

    a1, a2 = same_colored[:2]
    a3 = 2 * a2 - a1

    Possible_AP1 = [b1 * block_size + a1, b1 * block_size + a2, b1 * block_size + a3]
    Possible_AP2 = [b1 * block_size + a1, b2 * block_size + a2, b3 * block_size + a3]
    Possible_AP3 = [b1 * block_size + a3, b2 * block_size + a3, b3 * block_size + a3]

    bound1 = max(block_1_nums)
    bound2 = max(block_2_nums)
    bound3 = max(block_3_nums)
    lbound1 = min(block_1_nums)
    lbound2 = min(block_2_nums)
    lbound3 = min(block_3_nums)

    print(f"\nLet's look at block {b1+1}.\n")
    display_coloring_with_AP(Coloring, highlighted_AP=[], bound=bound1, block_size=block_size,
                             blocks_per_row=blocks_per_row, print_base=True, lower_bound=lbound1)
    
    if Coloring[Possible_AP1[2] - 1] == Coloring[Possible_AP1[0] - 1]:
        print(f"\nBlock {b1+1} already contains a monochromatic AP: {Possible_AP1}\n")
        display_coloring_with_AP(Coloring, highlighted_AP=Possible_AP1, bound=bound1,
                                 block_size=block_size, blocks_per_row=blocks_per_row, print_base=False, 
                                 lower_bound=lbound1)
    else:
        print(f"\nWithin block {b1+1}, the numbers {Possible_AP1[0]} and {Possible_AP1[1]} are both",
              f"{colorname_dict(Coloring[Possible_AP1[0]-1])}.\nBut {Possible_AP1[2]} is",
              f"{colorname_dict(Coloring[Possible_AP1[2]-1])}, so {Possible_AP1} is not a monochromatic AP", 
              f"in block {b1+1}.\n")
        time.sleep(5)
        print(f"In this coloring, block {b2+1} has the same color pattern as block {b1+1}\n")
        display_coloring_with_AP(Coloring, highlighted_AP=[], bound=bound2,block_size=block_size, 
                                 blocks_per_row=blocks_per_row, print_base=True, lower_bound=lbound2)
        time.sleep(5)
        print(f"\nIt seems like {block_size*b1+a1} and {block_size*b2+a2} could form a monochromatic {prog_len}-AP with",
              f"step size {block_size*b2+a2-(block_size*b1+a1)}, if {block_size*b3+a3} in block {b3}",
              f"is also {colorname_dict(Coloring[Possible_AP1[1]-1])}. \nLet's see... \n")

        if Coloring[Possible_AP2[2] - 1] == Coloring[Possible_AP2[0] - 1]:
            time.sleep(5)
            display_coloring_with_AP(Coloring, highlighted_AP=[], bound=bound3,block_size=block_size, 
                                 blocks_per_row=blocks_per_row, print_base=True, lower_bound=lbound3)
            print(f"\nIt is! We found our AP: {Possible_AP2}\n")
            display_coloring_with_AP(Coloring, highlighted_AP=Possible_AP2, bound=bound,
                                     block_size=block_size, blocks_per_row=blocks_per_row, print_base=False)
        else:
            display_coloring_with_AP(Coloring, highlighted_AP=[], bound=bound3,
                                     block_size=block_size, blocks_per_row=blocks_per_row, print_base=True, 
                                     lower_bound=lbound3)
            print(f"\nOh no :( {block_size*b3+a3} is {colorname_dict(Coloring[Possible_AP3[2]-1])}.")
            time.sleep(5)
            print(f"\nBut wait.",
                  f"Remember how {block_size*b1+a3} was {colorname_dict(Coloring[Possible_AP3[2]-1])} in",
                  f"block {b1+1}? That means {block_size*b2+a3} must also be",
                  f"{colorname_dict(Coloring[Possible_AP3[2]-1])} in block {b2+1} ")
            time.sleep(5)
            print(f"We found one! {Possible_AP3} is a {prog_len}-AP with step size {Possible_AP3[1]-Possible_AP3[0]}\n")
            display_coloring_with_AP(Coloring, highlighted_AP=Possible_AP3, bound=bound,
                                     block_size=block_size, blocks_per_row=blocks_per_row, print_base=False)


### Bad Combinatorially Derived Bounds 

We will now look at an example of the proof that $W(2,3)<325$ and why that is not particularly insightful.

In [4]:
bound = 325
prog_len = 3
num_colors = 2
block_size = 5
blocks_per_row = 5
num_per_row = block_size * blocks_per_row
num_blocks = int(bound / block_size)

seed0 = None 
seed1 = 2    # Case 1
seed2 = 5    # Case 2
seed3 = 1    # Case 3

Coloring, mono_APs = get_monochromatic_APs(bound, prog_len, num_colors, Coloring=None, seed=seed3)

display_coloring_with_AP(Coloring, highlighted_AP=[], bound=bound, block_size=block_size, 
                         blocks_per_row=blocks_per_row, print_base=True)
analyze_repeated_block_AP(Coloring, bound, block_size, blocks_per_row, num_colors, prog_len)

print(f"\nThis coloring of {bound} actually contains {len(mono_APs)} monochromatic {prog_len}-APs. \nHere is a random \
selection of other monochromatic {prog_len}-APs found in the above coloring:")

    1   2   3   4   5     6   7   8   9  10    11  12  13  14  15    16  17  18  19  20    21  22  23  24  25 
   26  27  28  29  30    31  32  33  34  35    36  37  38  39  40    41  42  43  44  45    46  47  48  49  50 
   51  52  53  54  55    56  57  58  59  60    61  62  63  64  65    66  67  68  69  70    71  72  73  74  75 
   76  77  78  79  80    81  82  83  84  85    86  87  88  89  90    91  92  93  94  95    96  97  98  99 100 
  101 102 103 104 105   106 107 108 109 110   111 112 113 114 115   116 117 118 119 120   121 122 123 124 125 
  126 127 128 129 130   131 132 133 134 135   136 137 138 139 140   141 142 143 144 145   146 147 148 149 150 
  151 152 153 154 155   156 157 158 159 160   161 162 163 164 165   166 167 168 169 170   171 172 173 174 175 
  176 177 178 179 180   181 182 183 184 185   186 187 188 189 190   191 192 193 194 195   196 197 198 199 200 
  201 202 203 204 205   206 207 208 209 210   211 212 213 214 215   216 217 218 219 220   221 222 223 224 225 
 

In [5]:
random_selection = np.random.choice(range(0, len(mono_APs)), size=50, replace=False)

wait_time = 0.5
for i in random_selection:
    ap = mono_APs[i]
    display_coloring_with_AP(Coloring, highlighted_AP=ap, bound=bound, block_size=block_size, 
                         blocks_per_row=blocks_per_row, print_base=False, print_stats=True)
    time.sleep(wait_time)
    clear_output(wait=True)


AP: [68, 172, 276] (step size = 104)

    1   2   3   4   5     6   7   8   9  10    11  12  13  14  15    16  17  18  19  20    21  22  23  24  25 
   26  27  28  29  30    31  32  33  34  35    36  37  38  39  40    41  42  43  44  45    46  47  48  49  50 
   51  52  53  54  55    56  57  58  59  60    61  62  63  64  65    66  67  68  69  70    71  72  73  74  75 
   76  77  78  79  80    81  82  83  84  85    86  87  88  89  90    91  92  93  94  95    96  97  98  99 100 
  101 102 103 104 105   106 107 108 109 110   111 112 113 114 115   116 117 118 119 120   121 122 123 124 125 
  126 127 128 129 130   131 132 133 134 135   136 137 138 139 140   141 142 143 144 145   146 147 148 149 150 
  151 152 153 154 155   156 157 158 159 160   161 162 163 164 165   166 167 168 169 170   171 172 173 174 175 
  176 177 178 179 180   181 182 183 184 185   186 187 188 189 190   191 192 193 194 195   196 197 198 199 200 
  201 202 203 204 205   206 207 208 209 210   211 212 213 214 215   216 2

### Demonstrating that $W(2,3)>8$ 

In [10]:
bound = 8
prog_len = 3
num_colors = 2
block_size = 4
blocks_per_row = 1
num_per_row = block_size * blocks_per_row
num_blocks = int(bound / block_size)

coloring = np.array([0, 1, 1, 0, 0, 1, 1, 0])
Coloring, mono_APs = get_monochromatic_APs(bound, prog_len, num_colors, Coloring=coloring, seed=None)
possible_aps = make_AP(1,bound,prog_len)
wait_time = 0.5

In [14]:
bound = 8
prog_len = 3
num_colors = 2
block_size = 4
blocks_per_row = 1
num_per_row = block_size * blocks_per_row
num_blocks = int(bound / block_size)

coloring = np.array([0, 1, 1, 0, 0, 1, 1, 0])
Coloring, mono_APs = get_monochromatic_APs(bound, prog_len, num_colors, Coloring=coloring, seed=None)
print(f"\nLet's see how many monochromatic {prog_len}-APs this coloring of {bound} has")
display_coloring_with_AP(Coloring, highlighted_AP=[], bound=bound, block_size=block_size, 
                             blocks_per_row=blocks_per_row, print_base=True, print_stats=False)

clear_output(wait=True) 
possible_aps = make_AP(1,bound,prog_len)
wait_time = 0.5

time.sleep(5)
clear_output(wait=True) 

for i in range(0,len(possible_aps)):
    color_pattern = [possible_aps[i][0]-1, possible_aps[i][1]-1, possible_aps[i][2]-1]
    cnames_inds = [Coloring[color_pattern[0]],Coloring[color_pattern[1]], Coloring[color_pattern[2]]]
    cnames = [colorname_dict(cnames_inds[i]) for i in [0,1,2]]
    display_coloring_with_AP(Coloring, highlighted_AP=possible_aps[i], bound=bound, block_size=block_size, 
                             blocks_per_row=blocks_per_row, print_base=False, print_stats=True)
    print(f"\n  Pattern: {cnames}")
    time.sleep(wait_time)
    clear_output(wait=True)
    
time.sleep(5)
clear_output(wait=True)    

display_coloring_with_AP(Coloring, highlighted_AP=[], bound=bound, block_size=block_size, 
                             blocks_per_row=blocks_per_row, print_base=True, print_stats=True)
print(f"\nThis coloring of {bound} contains {len(mono_APs)} monochromatic {prog_len}-APs.")

coloring9 = np.array([0, 1, 1, 0, 0, 1, 1, 0, 1])
Coloring, mono_APs = get_monochromatic_APs(bound+1, 3, 2, Coloring=coloring9, seed=None)

print(f"\nBut this nearly identical coloring of {bound+1} contains {len(mono_APs)} monochromatic {prog_len}-AP(s)\n")

print(mono_APs[0])

print()
display_coloring_with_AP(Coloring, highlighted_AP=mono_APs[0], bound=bound+1, block_size=block_size, 
                             blocks_per_row=1, print_base=False, print_stats=False)

  1 2 3 4 
  5 6 7 8 

This coloring of 8 contains 0 monochromatic 3-APs.

But this nearly identical coloring of 9 contains 1 monochromatic 3-AP(s)

[3, 6, 9]

  1 2 3 4 
  5 6 7 8 
  9 


### $W(2,3)=9$ holds for arbitrary colorings

In [8]:
bound = 9
prog_len = 3
num_colors = 2
block_size = 3
blocks_per_row = 1
num_per_row = block_size * blocks_per_row
num_blocks = int(bound / block_size)

print(f"\nLet's see how many monochromatic {prog_len}-APs this coloring of {bound} has\n")
Coloring, mono_APs = get_monochromatic_APs(bound, prog_len, num_colors, Coloring=None, seed=None)
display_coloring_with_AP(Coloring, highlighted_AP=[], bound=bound, block_size=block_size, 
                         blocks_per_row=blocks_per_row, print_base=True, print_stats=False)

time.sleep(5)
clear_output(wait=True)
wait_time = 1
for m in mono_APs:
    display_coloring_with_AP(Coloring, highlighted_AP=m, bound=bound, block_size=block_size, 
                         blocks_per_row=blocks_per_row, print_base=False, print_stats=True)
    time.sleep(wait_time)
    clear_output(wait=True)

display_coloring_with_AP(Coloring, highlighted_AP=[], bound=bound, block_size=block_size, 
                         blocks_per_row=blocks_per_row, print_base=True, print_stats=False)
print(f"\nThis coloring of {bound} contains {len(mono_APs)} monochromatic {prog_len}-APs:")
print(mono_APs)

  1 2 3 
  4 5 6 
  7 8 9 

This coloring of 9 contains 4 monochromatic 3-APs:

[[1, 4, 7], [2, 4, 6], [4, 5, 6], [5, 6, 7]]


## Demonstrating that $W(3,3)>26$ 

In [13]:
bound = 26
prog_len = 3
num_colors = 3
block_size = 13
blocks_per_row = 1
num_per_row = block_size * blocks_per_row
num_blocks = int(bound / block_size)

coloring = np.array([0, 0, 2, 2, 0, 0, 2, 1, 2, 1, 1, 0, 1, 0, 0, 2, 0, 2,2, 1, 0, 1, 1, 2, 1, 2])
Coloring, mono_APs = get_monochromatic_APs(bound, prog_len, num_colors, Coloring=coloring, seed=None)
print(f"\nLet's see how many monochromatic {prog_len}-APs this coloring of {bound} has:\n")
display_coloring_with_AP(Coloring, highlighted_AP=[], bound=bound, block_size=block_size, 
                             blocks_per_row=blocks_per_row, print_base=True, print_stats=False)

time.sleep(5)
clear_output(wait=True)
possible_aps = make_AP(1,26,3)
wait_time = 0.15

for i in range(0,len(possible_aps)):
    color_pattern = [possible_aps[i][0]-1, possible_aps[i][1]-1, possible_aps[i][2]-1]
    cnames_inds = [Coloring[color_pattern[0]],Coloring[color_pattern[1]], Coloring[color_pattern[2]]]
    cnames = [colorname_dict(cnames_inds[0]),colorname_dict(cnames_inds[1]), colorname_dict(cnames_inds[2])]
    display_coloring_with_AP(Coloring, highlighted_AP=possible_aps[i], bound=bound, block_size=block_size, 
                             blocks_per_row=blocks_per_row, print_base=False, print_stats=True)
    print(f"\n  Pattern: {cnames}")
    time.sleep(wait_time)
    clear_output(wait=True)
    
time.sleep(5)
clear_output(wait=True)    

print(f"\nThis coloring of {bound} contains {len(mono_APs)} monochromatic {prog_len}-APs:\n")

display_coloring_with_AP(Coloring, highlighted_AP=[], bound=bound, block_size=block_size, 
                             blocks_per_row=blocks_per_row, print_base=True, print_stats=True)

coloring = np.array([0, 0, 2, 2, 0, 0, 2, 1, 2, 1, 1, 0, 1, 0, 0, 2, 0, 2,2, 1, 0, 1, 1, 2, 1, 2, 2])
Coloring, mono_APs = get_monochromatic_APs(bound+1, prog_len, num_colors, Coloring=coloring, seed=None)

print(f"\nThis nearly identical coloring of {bound+1} contains {len(mono_APs)} monochromatic {prog_len}-AP(s):")
print(mono_APs[0])

print()
display_coloring_with_AP(Coloring, highlighted_AP=mono_APs[0], bound=bound+1, block_size=block_size, 
                             blocks_per_row=1, print_base=False, print_stats=False)




This coloring of 26 contains 0 monochromatic 3-APs:

   1  2  3  4  5  6  7  8  9 10 11 12 13 
  14 15 16 17 18 19 20 21 22 23 24 25 26 

This nearly identical coloring of 27 contains 1 monochromatic 3-AP(s):
[9, 18, 27]

   1  2  3  4  5  6  7  8  9 10 11 12 13 
  14 15 16 17 18 19 20 21 22 23 24 25 26 
  27 
